# How to get reverse kinetics for a rxn?

This ipynb will solve the commonly seen question, what is the reverse reaction rate at certain condition given the forward (A, n, Ea) kinetics.

- input: reactants and products of forward reaction, forward kinetics (A, n, Ea), temperature of interest

- output: reverse reaction rate at input temperature

## Inputs

In [1]:
# reactants and products
reactants = ['CC','CC[CH2]']
products = ['C[CH2]','CCC']

In [2]:
# forward kinetics (A, n, Ea)
A = 1e11 # cm3/mol/s
n = 0.0
Ea = 10.4 # kcal/mol

In [3]:
# temperature of interest
T = 1000 # K

## Calculation procedure

In [4]:
from rmgpy.rmg.model import Species
from rmgpy.data.thermo import ThermoDatabase, NASA
import math
R = 8.314
thermodb = ThermoDatabase()
path = '/Users/kehang/Code/rmgmaster/RMG-database/input/thermo'
thermodb.load(path)

In [5]:
# get Gibbs free energy for all the species
rxts_prds_spc = []
for smiles in (reactants + products):
    spc = Species(label=smiles).fromSMILES(smiles)
    rxts_prds_spc.append(spc)
    # calculate thermo
    thermo = thermodb.getThermoData(spc, None, None)
    _ = spc.processThermoData(None, thermo, NASA)

In [6]:
# get deltaG for the forward reaction
deltaG = 0
for spc in rxts_prds_spc:
    if spc.label in reactants:
        deltaG -= spc.getFreeEnergy(T)
    elif spc.label in products:
        deltaG += spc.getFreeEnergy(T)      

In [7]:
# calculate equilibrium const K
K = math.exp(-deltaG/R/T)

# calculate forward k_f
k_f = A*(T**n)*math.exp(-Ea*4184/R/T)

# calculate reverse k_r by k_f/K
k_r = k_f/K

Next steps:

- the equilibrium const need more investigation when the numbers of products and reactants are not equal